# AutoGen RoundRobinGroupChat
A team is a group of agents that work together to achieve a common goal.


## Load Azure Configurations

In [1]:
from dotenv import load_dotenv
import os

azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_openai_deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
azure_openai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")

## Create Azure OpenAI Client
Using the model client class

In [2]:
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

# Create the token provider
#token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")

az_model_client = AzureOpenAIChatCompletionClient(
    azure_deployment=azure_openai_deployment,
    model=azure_openai_deployment,
    api_version=azure_openai_api_version,
    azure_endpoint=azure_openai_endpoint,
    # azure_ad_token_provider=token_provider,  # Optional if you choose key-based authentication.
    api_key=azure_openai_key, # For key-based authentication.
)

## Creating a Team

RoundRobinGroupChat is a team configuration where all agents share the same context and take turns responding in a round-robin fashion. Each agent, during its turn, broadcasts its response to all other agents, ensuring that the entire team maintains a consistent context.

We create a team with two AssistantAgent and a TextMentionTermination condition that stops the team when a specific word is detected in the agent’s response.

In [3]:
import asyncio

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.base import TaskResult
from autogen_agentchat.conditions import ExternalTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_core import CancellationToken


# Create the primary agent.
primary_agent = AssistantAgent(
    "primary",
    model_client=az_model_client,
    system_message="You are a helpful AI assistant.",
)

# Create the critic agent.
critic_agent = AssistantAgent(
    "critic",
    model_client=az_model_client,
    system_message="Provide constructive feedback. Respond with 'APPROVE' to when your feedbacks are addressed.",
)

# Define a termination condition that stops the task if the critic approves.
text_termination = TextMentionTermination("APPROVE")

# Create a team with the primary and critic agents.
team = RoundRobinGroupChat([primary_agent, critic_agent], termination_condition=text_termination)

## Running a Team
Call the run() method to start the team with a task.

The termination condition was met when the word “APPROVE” is detected in the agent’s response. When the team stops, it returns a TaskResult object with all the messages produced by the agents in the team.

In [4]:
# When running inside a script, use a async main function and call it from `asyncio.run(...)`.
result = await team.run(task="Write a short poem about the fall season.")
print(result)

/home/codespace/.python/current/lib/python3.12/site-packages/autogen_agentchat/agents/_assistant_agent.py:343: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-05-13. Model mapping may be incorrect.
  result = await self._model_client.create(


TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Write a short poem about the fall season.', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=28, completion_tokens=123), content="Leaves of amber, gold, and red,\nFluttering whispers overhead,\nCrisp air paints the twilight skies,\nAs autumn’s gentle breath complies.\n\nPumpkin patches, harvest moon,\nRustling woods in quiet tune,\nSweaters, scarves, and boots appear,\nHeralding this time of year.\n\nBonfires warm with crackling glow,\nStories shared as embers grow,\nNature’s palette, rich and bright,\nGuides us to the sleepy night.\n\nIn fall’s embrace, we find retreat,\nA soothing rhythm, bittersweet,\nFor as the fleeting moments pass,\nWe cherish autumn's gentle grasp.", type='TextMessage'), TextMessage(source='critic', models_usage=RequestUsage(prompt_tokens=168, completion_tokens=190), content='Your poem beautifully captures the essence of the fall season with vivi

## Observing a Team
Similar to the agent’s on_messages_stream() method, you can stream the team’s messages while it is running by calling the run_stream() method. This method returns a generator that yields messages produced by the agents in the team as they are generated, with the final item being the TaskResult object.

In [5]:
# When running inside a script, use a async main function and call it from `asyncio.run(...)`.
await team.reset()  # Reset the team for a new task.
async for message in team.run_stream(task="Write a short poem about the fall season."):
    if isinstance(message, TaskResult):
        print("Stop Reason:", message.stop_reason)
    else:
        print(message)

source='user' models_usage=None content='Write a short poem about the fall season.' type='TextMessage'
source='primary' models_usage=RequestUsage(prompt_tokens=28, completion_tokens=96) content="Golden leaves in crisp air twirl,\nAutumn’s dance, a whispered whirl.\nPumpkin patches, orchard’s yield,\nHarvest bounty, nature’s shield.\n\nMoments soft in amber light,\nSweater wrapped in cozy night.\nBonfires blaze with stories told,\nWarmth of hearts as nights grow cold.\n\nGeese in flight, a southward stream,\nMisty mornings, nature's dream.\nFall's embrace, a fleeting sigh,\nBeauty’s breath as trees comply." type='TextMessage'
source='critic' models_usage=RequestUsage(prompt_tokens=141, completion_tokens=152) content='What a beautifully crafted poem capturing the essence of fall!\n\n1. The imagery evokes the vivid colors and atmosphere of the season splendidly.\n2. The rhythm and rhyme create a soothing flow that enhances the reading experience.\n3. The progression from describing the en

## Console()
The Console() method provides a convenient way to print messages to the console with proper formatting.

In [6]:
await team.reset()  # Reset the team for a new task.
await Console(team.run_stream(task="Write a short poem about the fall season."))  # Stream the messages to the console.

---------- user ----------
Write a short poem about the fall season.
---------- primary ----------
Golden leaves in crisp air dance,
Whispers of an autumn trance.
Pumpkin scents and apple treats,
Crunching paths beneath our feet.

Sweaters woven, fires warm,
Nature dons her twilight form.
Crimson, amber, woodland hue,
Painting skies a softer blue.

Mornings draped in misty veils,
Evening's hush as daylight pales.
Time of harvest, heart's delight,
Fall's embrace, the world's goodnight.
[Prompt tokens: 28, Completion tokens: 92]
---------- critic ----------
Your poem beautifully captures the essence of fall! Your imagery of golden leaves, pumpkin scents, and crisp air evokes the sensory experience of the season. The flow and rhythm of your verses are pleasant and engaging. The vivid descriptions of autumn's transition and embrace are well-executed.

However, if you would like to enhance your poem even further, consider adding a personal touch or a unique perspective, such as how fall per

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Write a short poem about the fall season.', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=28, completion_tokens=92), content="Golden leaves in crisp air dance,\nWhispers of an autumn trance.\nPumpkin scents and apple treats,\nCrunching paths beneath our feet.\n\nSweaters woven, fires warm,\nNature dons her twilight form.\nCrimson, amber, woodland hue,\nPainting skies a softer blue.\n\nMornings draped in misty veils,\nEvening's hush as daylight pales.\nTime of harvest, heart's delight,\nFall's embrace, the world's goodnight.", type='TextMessage'), TextMessage(source='critic', models_usage=RequestUsage(prompt_tokens=137, completion_tokens=128), content="Your poem beautifully captures the essence of fall! Your imagery of golden leaves, pumpkin scents, and crisp air evokes the sensory experience of the season. The flow and rhythm of your verses are pleasant and engaging.

## Team has a State

In [7]:
await Console(team.run_stream(task="What was the last message?"))  # Stream the messages to the console.

---------- user ----------
What was the last message?
---------- primary ----------
The last message was a positive critique and approval of your revised version of the fall season poem. It highlighted how your additions of morning drives, children's laughter, and Grandma's kitchen added a personal and cozy touch to the poem, deepening the emotional connection for the reader. The critique also praised the balance between universal autumn experiences and personal memories, maintaining the beautiful flow and vivid descriptions from the original poem. The overall feedback was very approving and complimentary of your work.
[Prompt tokens: 509, Completion tokens: 93]
---------- critic ----------
APPROVE
[Prompt tokens: 619, Completion tokens: 4]
---------- Summary ----------
Number of messages: 3
Finish reason: Text 'APPROVE' mentioned
Total prompt tokens: 1128
Total completion tokens: 97
Duration: 1.54 seconds


TaskResult(messages=[TextMessage(source='user', models_usage=None, content='What was the last message?', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=509, completion_tokens=93), content="The last message was a positive critique and approval of your revised version of the fall season poem. It highlighted how your additions of morning drives, children's laughter, and Grandma's kitchen added a personal and cozy touch to the poem, deepening the emotional connection for the reader. The critique also praised the balance between universal autumn experiences and personal memories, maintaining the beautiful flow and vivid descriptions from the original poem. The overall feedback was very approving and complimentary of your work.", type='TextMessage'), TextMessage(source='critic', models_usage=RequestUsage(prompt_tokens=619, completion_tokens=4), content='APPROVE', type='TextMessage')], stop_reason="Text 'APPROVE' mentioned")

## Resetting a Team
You can reset the team by calling the reset() method. This method will clear the team’s state, including all agents. It will call the each agent’s on_reset() method

In [8]:
await team.reset()  # Reset the team for the next run.

## Stopping a Team
You can also stop the team from outside by using the ExternalTermination.

Calling set() on ExternalTermination will stop the team when the current agent’s turn is over. 

Thus, the team may not stop immediately. This allows the current agent to finish its turn and broadcast the final message to the team before the team stops, keeping the team’s state consistent.

In [9]:
# Create a new team with an external termination condition.
external_termination = ExternalTermination()
team = RoundRobinGroupChat(
    [primary_agent, critic_agent],
    termination_condition=external_termination | text_termination,  # Use the bitwise OR operator to combine conditions.
)

# Run the team in a background task.
run = asyncio.create_task(Console(team.run_stream(task="Write a short poem about the fall season.")))

# Wait for some time.
await asyncio.sleep(0.1)

# Stop the team.
external_termination.set()

# Wait for the team to finish.
await run # Primary agent is still allowed to complete its task.

---------- user ----------
Write a short poem about the fall season.
---------- primary ----------
Leaves descend in a golden dance,
Whispers of autumn's sweet romance.
A tapestry of crimson and gold,
Stories of summer softly told.

Crisp air carries distant chimes,
Harvest moons and apple climbs.
Warmth retreats, yet hearts ignite,
In the embrace of fall's delight.

Pumpkin patches, fireside tales,
Rustling woods and winding trails.
Nature's quilt in vibrant hue,
A fleeting time, yet born anew.

Through swirling winds and fading light,
Autumn weaves the day to night.
A gentle pause before the snow,
In fall's embrace, the world aglow.
[Prompt tokens: 28, Completion tokens: 123]
---------- Summary ----------
Number of messages: 2
Finish reason: External termination requested
Total prompt tokens: 28
Total completion tokens: 123
Duration: 1.96 seconds


TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Write a short poem about the fall season.', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=28, completion_tokens=123), content="Leaves descend in a golden dance,\nWhispers of autumn's sweet romance.\nA tapestry of crimson and gold,\nStories of summer softly told.\n\nCrisp air carries distant chimes,\nHarvest moons and apple climbs.\nWarmth retreats, yet hearts ignite,\nIn the embrace of fall's delight.\n\nPumpkin patches, fireside tales,\nRustling woods and winding trails.\nNature's quilt in vibrant hue,\nA fleeting time, yet born anew.\n\nThrough swirling winds and fading light,\nAutumn weaves the day to night.\nA gentle pause before the snow,\nIn fall's embrace, the world aglow.", type='TextMessage')], stop_reason='External termination requested')

## Resuming a Team
Teams are stateful and maintains the conversation history and context after each run, unless you reset the team.

You can resume a team to continue from where it left off by calling the run() or run_stream() method again without a new task. RoundRobinGroupChat will continue from the next agent in the round-robin order.

In this example, critic agent continued where primary agent left off

In [10]:
await Console(team.run_stream())  # Resume the team to continue the last task.

---------- critic ----------
Your poem beautifully captures the essence of the fall season. Here are some areas where you might enhance the imagery and rhythm:

1. The first stanza sets a lovely scene but might benefit from a more unique phrasing. Consider reworking the second line for even more impact.

2. The second stanza is quite charming but the last line could be revised to enhance the meter. Perhaps something like: "In fall's gentle, glowing light."

3. The third stanza starts strong with imagery, but the last two lines feel a bit rushed. Maybe add more descriptive words or slow the pace slightly.

4. The final stanza has a serene mood, but consider a stronger closing image to leave a lasting impression.

Overall, these suggestions are merely to fine-tune an already beautiful piece. You've captured the warmth and beauty of the fall season admirably. 

Please revise and I am sure it will be fantastic.
[Prompt tokens: 168, Completion tokens: 181]
---------- primary ----------
Leav

TaskResult(messages=[TextMessage(source='critic', models_usage=RequestUsage(prompt_tokens=168, completion_tokens=181), content='Your poem beautifully captures the essence of the fall season. Here are some areas where you might enhance the imagery and rhythm:\n\n1. The first stanza sets a lovely scene but might benefit from a more unique phrasing. Consider reworking the second line for even more impact.\n\n2. The second stanza is quite charming but the last line could be revised to enhance the meter. Perhaps something like: "In fall\'s gentle, glowing light."\n\n3. The third stanza starts strong with imagery, but the last two lines feel a bit rushed. Maybe add more descriptive words or slow the pace slightly.\n\n4. The final stanza has a serene mood, but consider a stronger closing image to leave a lasting impression.\n\nOverall, these suggestions are merely to fine-tune an already beautiful piece. You\'ve captured the warmth and beauty of the fall season admirably. \n\nPlease revise an

## Aborting a Team
You can abort a call to run() or run_stream() during execution by setting a CancellationToken passed to the cancellation_token parameter.

Different from stopping a team, aborting a team will immediately stop the team and raise a CancelledError exception.

In [11]:
# Create a cancellation token.
cancellation_token = CancellationToken()

# Use another coroutine to run the team.
run = asyncio.create_task(
    team.run(
        task="Translate the poem to Spanish.",
        cancellation_token=cancellation_token,
    )
)

# Cancel the run.
cancellation_token.cancel()

try:
    result = await run  # This will raise a CancelledError.
except asyncio.CancelledError:
    print("Task was cancelled.")

Task was cancelled.


## Other Built-In Termination Conditions:
- MaxMessageTermination: Stops after a specified number of messages have been produced, including both agent and task messages.
- TextMentionTermination: Stops when specific text or string is mentioned in a message (e.g., “TERMINATE”).
- ExternalTermination: Enables programmatic control of termination from outside the run. This is useful for UI integration (e.g., “Stop” buttons in chat interfaces).
- TokenUsageTermination: Stops when a certain number of prompt or completion tokens are used. This requires the agents to report token usage in their messages.
- TimeoutTermination: Stops after a specified duration in seconds.
- SourceMatchTermination: Stops after a specific agent responds.
- HandoffTermination: Stops when a handoff to a specific target is requested. Handoff messages can be used to build patterns such as Swarm. This is useful when you want to pause the run and allow application or user to provide input when an agent hands off to them.
- StopMessageTermination: Stops when a StopMessage is produced by an agent.

In [12]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination, ExternalTermination, TokenUsageTermination, TimeoutTermination, SourceMatchTermination, HandoffTermination, StopMessageTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console

# Create the first agent.
first_agent = AssistantAgent(
    "first",
    model_client=az_model_client,
    system_message="You will give a random number between 1 and 10.",
)

# Create the second agent.
second_agent = AssistantAgent(
    "second",
    model_client=az_model_client,
    system_message="You will give a random number between 1 and 10.",
)

# Create the second agent.
third_agent = AssistantAgent(
    "third",
    model_client=az_model_client,
    system_message="You will give a random number between 1 and 10.",
)

# MaxMessageTermination
termination = MaxMessageTermination(5) # terminate after 5 messages
team = RoundRobinGroupChat([first_agent, second_agent, third_agent], termination_condition=termination)

await Console(team.run_stream(task="Give a number"))

---------- user ----------
Give a number
---------- first ----------
Sure! How about the number 7?
[Prompt tokens: 28, Completion tokens: 10]
---------- second ----------
Let's go with the number 3.
[Prompt tokens: 42, Completion tokens: 9]
---------- third ----------
I'll choose the number 8 for you.
[Prompt tokens: 56, Completion tokens: 10]
---------- first ----------
All right, you got the number 8!
[Prompt tokens: 71, Completion tokens: 11]
---------- Summary ----------
Number of messages: 5
Finish reason: Maximum number of messages 5 reached, current message count: 5
Total prompt tokens: 197
Total completion tokens: 40
Duration: 1.68 seconds


TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Give a number', type='TextMessage'), TextMessage(source='first', models_usage=RequestUsage(prompt_tokens=28, completion_tokens=10), content='Sure! How about the number 7?', type='TextMessage'), TextMessage(source='second', models_usage=RequestUsage(prompt_tokens=42, completion_tokens=9), content="Let's go with the number 3.", type='TextMessage'), TextMessage(source='third', models_usage=RequestUsage(prompt_tokens=56, completion_tokens=10), content="I'll choose the number 8 for you.", type='TextMessage'), TextMessage(source='first', models_usage=RequestUsage(prompt_tokens=71, completion_tokens=11), content='All right, you got the number 8!', type='TextMessage')], stop_reason='Maximum number of messages 5 reached, current message count: 5')

In [13]:
#TextMentionTermination
termination = TextMentionTermination("4") # Terminate when the message contains "4".
team = RoundRobinGroupChat([first_agent, second_agent, third_agent], termination_condition=termination)
await Console(team.run_stream(task="Give a number"))

---------- user ----------
Give a number
---------- first ----------
How about the number 5?
[Prompt tokens: 94, Completion tokens: 8]
---------- second ----------
Let's go with the number 8.
[Prompt tokens: 76, Completion tokens: 9]
---------- third ----------
I'll choose the number 2 for you.
[Prompt tokens: 105, Completion tokens: 10]
---------- first ----------
Alright, the number is 2!
[Prompt tokens: 135, Completion tokens: 9]
---------- second ----------
Alright, I'll give you the number 6.
[Prompt tokens: 118, Completion tokens: 11]
---------- third ----------
How about the number 4?
[Prompt tokens: 149, Completion tokens: 8]
---------- Summary ----------
Number of messages: 7
Finish reason: Text '4' mentioned
Total prompt tokens: 677
Total completion tokens: 55
Duration: 2.50 seconds


TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Give a number', type='TextMessage'), TextMessage(source='first', models_usage=RequestUsage(prompt_tokens=94, completion_tokens=8), content='How about the number 5?', type='TextMessage'), TextMessage(source='second', models_usage=RequestUsage(prompt_tokens=76, completion_tokens=9), content="Let's go with the number 8.", type='TextMessage'), TextMessage(source='third', models_usage=RequestUsage(prompt_tokens=105, completion_tokens=10), content="I'll choose the number 2 for you.", type='TextMessage'), TextMessage(source='first', models_usage=RequestUsage(prompt_tokens=135, completion_tokens=9), content='Alright, the number is 2!', type='TextMessage'), TextMessage(source='second', models_usage=RequestUsage(prompt_tokens=118, completion_tokens=11), content="Alright, I'll give you the number 6.", type='TextMessage'), TextMessage(source='third', models_usage=RequestUsage(prompt_tokens=149, completion_tokens=8), conten

In [14]:
import asyncio
#ExternalTermination
termination = ExternalTermination() # Create an external termination condition.

team = RoundRobinGroupChat([first_agent, second_agent, third_agent], termination_condition=termination)

# Run the team in a background task.
run = asyncio.create_task(Console(team.run_stream(task="Give a number")))

# Wait for some time.
await asyncio.sleep(3)

# Stop the team.
termination.set()

# Wait for the team to finish.
await run


---------- user ----------
Give a number
---------- first ----------
Sure! How about the number 4?
[Prompt tokens: 156, Completion tokens: 10]
---------- second ----------
I suggest the number 9.
[Prompt tokens: 156, Completion tokens: 8]
---------- third ----------
Let's go with the number 1.
[Prompt tokens: 197, Completion tokens: 9]
---------- first ----------
Alright, you got the number 1!
[Prompt tokens: 197, Completion tokens: 10]
---------- second ----------
I'm giving you the number 10.
[Prompt tokens: 197, Completion tokens: 9]
---------- third ----------
I'll go with the number 7.
[Prompt tokens: 239, Completion tokens: 9]
---------- first ----------
Great choice! The number is 7.
[Prompt tokens: 239, Completion tokens: 10]
---------- second ----------
Let's go with the number 5.
[Prompt tokens: 239, Completion tokens: 9]
---------- Summary ----------
Number of messages: 9
Finish reason: External termination requested
Total prompt tokens: 1620
Total completion tokens: 74
Dura

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Give a number', type='TextMessage'), TextMessage(source='first', models_usage=RequestUsage(prompt_tokens=156, completion_tokens=10), content='Sure! How about the number 4?', type='TextMessage'), TextMessage(source='second', models_usage=RequestUsage(prompt_tokens=156, completion_tokens=8), content='I suggest the number 9.', type='TextMessage'), TextMessage(source='third', models_usage=RequestUsage(prompt_tokens=197, completion_tokens=9), content="Let's go with the number 1.", type='TextMessage'), TextMessage(source='first', models_usage=RequestUsage(prompt_tokens=197, completion_tokens=10), content='Alright, you got the number 1!', type='TextMessage'), TextMessage(source='second', models_usage=RequestUsage(prompt_tokens=197, completion_tokens=9), content="I'm giving you the number 10.", type='TextMessage'), TextMessage(source='third', models_usage=RequestUsage(prompt_tokens=239, completion_tokens=9), content="I

In [15]:
#TokenUsageTermination
termination = TokenUsageTermination(1000) # terminate when token usage is met.
team = RoundRobinGroupChat([first_agent, second_agent, third_agent], termination_condition=termination)
await Console(team.run_stream(task="Give a number"))

---------- user ----------
Give a number
---------- first ----------
Sure! How about the number 6?
[Prompt tokens: 261, Completion tokens: 10]
---------- second ----------
How about the number 3?
[Prompt tokens: 275, Completion tokens: 8]
---------- third ----------
Let's go with the number 5.
[Prompt tokens: 288, Completion tokens: 9]
---------- first ----------
Alright, the number is 5!
[Prompt tokens: 302, Completion tokens: 9]
---------- Summary ----------
Number of messages: 5
Finish reason: Token usage limit reached, total token count: 1162, prompt token count: 1126, completion token count: 36.
Total prompt tokens: 1126
Total completion tokens: 36
Duration: 1.57 seconds


TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Give a number', type='TextMessage'), TextMessage(source='first', models_usage=RequestUsage(prompt_tokens=261, completion_tokens=10), content='Sure! How about the number 6?', type='TextMessage'), TextMessage(source='second', models_usage=RequestUsage(prompt_tokens=275, completion_tokens=8), content='How about the number 3?', type='TextMessage'), TextMessage(source='third', models_usage=RequestUsage(prompt_tokens=288, completion_tokens=9), content="Let's go with the number 5.", type='TextMessage'), TextMessage(source='first', models_usage=RequestUsage(prompt_tokens=302, completion_tokens=9), content='Alright, the number is 5!', type='TextMessage')], stop_reason='Token usage limit reached, total token count: 1162, prompt token count: 1126, completion token count: 36.')

In [16]:
#TimeoutTermination
termination = TimeoutTermination(5) # terminate when time out is met.
team = RoundRobinGroupChat([first_agent, second_agent, third_agent], termination_condition=termination)
await Console(team.run_stream(task="Give a number"))

---------- user ----------
Give a number
---------- first ----------
Sure! How about the number 4?
[Prompt tokens: 323, Completion tokens: 10]
---------- second ----------
Let's go with the number 9.
[Prompt tokens: 310, Completion tokens: 9]
---------- third ----------
How about the number 2?
[Prompt tokens: 338, Completion tokens: 8]
---------- first ----------
Alright, the number is 2!
[Prompt tokens: 364, Completion tokens: 9]
---------- second ----------
Alright, I'll go with the number 8.
[Prompt tokens: 350, Completion tokens: 11]
---------- third ----------
How about the number 7?
[Prompt tokens: 380, Completion tokens: 8]
---------- first ----------
Sure, let's go with the number 7!
[Prompt tokens: 406, Completion tokens: 11]
---------- second ----------
Sure, let's go with the number 7!
[Prompt tokens: 394, Completion tokens: 11]
---------- third ----------
How about the number 3?
[Prompt tokens: 424, Completion tokens: 8]
---------- first ----------
Great choice! The number 

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Give a number', type='TextMessage'), TextMessage(source='first', models_usage=RequestUsage(prompt_tokens=323, completion_tokens=10), content='Sure! How about the number 4?', type='TextMessage'), TextMessage(source='second', models_usage=RequestUsage(prompt_tokens=310, completion_tokens=9), content="Let's go with the number 9.", type='TextMessage'), TextMessage(source='third', models_usage=RequestUsage(prompt_tokens=338, completion_tokens=8), content='How about the number 2?', type='TextMessage'), TextMessage(source='first', models_usage=RequestUsage(prompt_tokens=364, completion_tokens=9), content='Alright, the number is 2!', type='TextMessage'), TextMessage(source='second', models_usage=RequestUsage(prompt_tokens=350, completion_tokens=11), content="Alright, I'll go with the number 8.", type='TextMessage'), TextMessage(source='third', models_usage=RequestUsage(prompt_tokens=380, completion_tokens=8), content='

In [17]:
#SourceMatchTermination
termination = SourceMatchTermination("third") # terminate when source match agent is met.
team = RoundRobinGroupChat([first_agent, second_agent, third_agent], termination_condition=termination)
await Console(team.run_stream(task="Give a number"))

---------- user ----------
Give a number
---------- first ----------
Sure! How about the number 5?
[Prompt tokens: 515, Completion tokens: 10]
---------- second ----------
How about the number 8?
[Prompt tokens: 475, Completion tokens: 8]
---------- third ----------
I'll go with the number 1.
[Prompt tokens: 516, Completion tokens: 9]
---------- Summary ----------
Number of messages: 4
Finish reason: 'third' answered
Total prompt tokens: 1506
Total completion tokens: 27
Duration: 1.21 seconds


TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Give a number', type='TextMessage'), TextMessage(source='first', models_usage=RequestUsage(prompt_tokens=515, completion_tokens=10), content='Sure! How about the number 5?', type='TextMessage'), TextMessage(source='second', models_usage=RequestUsage(prompt_tokens=475, completion_tokens=8), content='How about the number 8?', type='TextMessage'), TextMessage(source='third', models_usage=RequestUsage(prompt_tokens=516, completion_tokens=9), content="I'll go with the number 1.", type='TextMessage')], stop_reason="'third' answered")